In [ ]:
!pip install findspark

In [2]:
import findspark
findspark.init()

from pyspark.sql import SparkSession
from pyspark.sql.functions import col, regexp_replace
from pyspark.sql.types import StringType

# Crear sesión de Spark
spark = SparkSession.builder.appName("LimpiezaMetro").getOrCreate()

In [3]:
# Leer el archivo CSV
df = spark.read.csv("./work/data/afluencia-metro.csv", header=True, inferSchema=True)

In [4]:
# Función para corregir la codificación de caracteres
def corregir_codificacion(df, column_name):
    return df.withColumn(column_name,
        regexp_replace(
            regexp_replace(
                regexp_replace(
                    regexp_replace(
                        regexp_replace(
                            regexp_replace(
                                regexp_replace(
                                    regexp_replace(
                                        regexp_replace(
                                            regexp_replace(
                                                col(column_name),
                                                    "Ã³", "ó"),
                                                "Ã¡", "á"),
                                            "Ã­", "í"),
                                        "Ã©", "é"),
                                    "Ãº", "ú"),
                                "Ã±", "ñ"),
                            "Ã±", "ñ"),
                        "Ã", "í"),
                    "Ã", "á"),
                "Ã", "ó")
    )

In [5]:
# Aplicar la corrección a la columna de estaciones
df_corregido = corregir_codificacion(df, "estacion")

In [6]:
# Eliminar "Linea " de la columna "linea" y convertir a texto
df_corregido = df_corregido.withColumn("linea", regexp_replace(col("linea"), "Linea ", "").cast(StringType()))


In [7]:
# Guardar el resultado
df_corregido.coalesce(1).write.csv("./work/data/afluencia-metro-corregido.csv", header=True)